## Random Forest Classifier CAT_DOG

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Data Processing
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [4]:
# Import necessary libraries
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# Load libraries
# import pandas as pd
# from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
# from sklearn.model_selection import train_test_split # Import train_test_split function
# from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [36]:
# load the datatset dog and cat
from Animal_Adoption.preproc.data_cat_or_dog import get_data
df= get_data('Cat')

KeyError: "['animal_type_x', 'breed_x', 'color_y', 'intake_condition', 'intake_type', 'sex_upon_intake', 'age_upon_intake_(years)', 'intake_datetime', 'time_in_shelter_days', 'sex_y', 'breed_y'] not in index"

## Balanced data check

In [13]:
#balenced dataset for the column 'age_upon_intake_(years)'
# relable the column 'age_upon_intake_(years)'
df['age_upon_intake_(years)'].value_counts()

1.000000     12351
2.000000     10682
0.082192      7419
3.000000      4935
0.164384      3975
4.000000      2995
5.000000      2701
0.076712      2501
0.057534      2261
0.328767      2000
0.246575      1900
6.000000      1841
0.410959      1697
7.000000      1567
0.038356      1546
8.000000      1514
0.493151      1311
0.019178      1172
0.739726      1162
10.000000     1149
0.575342      1120
0.657534       872
9.000000       863
0.821918       700
12.000000      590
0.904110       516
11.000000      449
0.000000       408
13.000000      378
0.008219       341
0.002740       332
0.005479       270
14.000000      249
15.000000      212
0.095890       199
0.016438       183
0.010959       171
0.013699       118
16.000000      106
17.000000       59
18.000000       26
19.000000       13
20.000000       12
22.000000        4
Name: age_upon_intake_(years), dtype: int64

In [14]:
#balenced dataset for the column 'intake_condition_2classes'
df['intake_condition_2classes'].value_counts()

normal        66962
not normal     7861
Name: intake_condition_2classes, dtype: int64

In [15]:
#balenced dataset for the column 'sex'
df['sex'].value_counts()

Male       37533
Female     34519
Unknown     2818
Name: sex, dtype: int64

In [16]:
#balenced dataset for the column 'sex_type'
df['sex_type'].value_counts()

Neutered    53631
Intact      18421
Unknown      2818
Name: sex_type, dtype: int64

In [17]:
#balenced dataset for the column 'animal_type'
df['animal_type'].value_counts()

Dog    45350
Cat    29520
Name: animal_type, dtype: int64

In [18]:
#balenced dataset for the column 'breed_2classes'
df['breed_2classes'].value_counts()

mixed        71581
not mixed     3289
Name: breed_2classes, dtype: int64

In [19]:
#balenced dataset for the column 'color_3classes'
df['color_3classes'].value_counts()

Monocolor    39716
Bicolor      31243
Tricolor      3911
Name: color_3classes, dtype: int64

In [20]:
#balenced dataset for the column 'outcome_type_2classes'
df['outcome_type_2classes'].value_counts()

adopted        56548
not adopted    18322
Name: outcome_type_2classes, dtype: int64

In [21]:
#balenced dataset for the column 'outcome_type_2classes'
df['time_in_shelter_days_round_5classes'].value_counts()

between 1 and 5 days      29492
between 6 and 10 days     12233
higher than 30 days       11591
several hours              9869
between 11 and 15 days     5295
between 16 and 20 days     2665
between 21 and 25 days     2002
between 26 and 30 days     1723
Name: time_in_shelter_days_round_5classes, dtype: int64

## TRAIN_TEST_DATASET- 8 classes

In [54]:
# I have to scale them? 

# Split the data into features (X) and target (y)
X = df[['intake_condition_2classes', 'animal_type', 'sex', 'sex_type', 'breed_2classes', 'color_3classes',\
    'outcome_type_2classes']]

y = df['time_in_shelter_days_round_8classes'] # next one with 'time_in_shelter_days_round_5classes', 'time_in_shelter_days_round_2classes'


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [55]:
# create a list of numerical columns and categorical columns
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = []

categorical_features = ['intake_condition_2classes', 'animal_type', 'sex', 'sex_type', 'breed_2classes', 'color_3classes',\
    'outcome_type_2classes']

# create the transformer for numerical columns
numeric_transformer = make_pipeline(StandardScaler())

# create the transformer for categorical columns
categorical_transformer = make_pipeline(OneHotEncoder(sparse=False, handle_unknown='ignore'))


# use ColumnTransformer to specify which columns need to be preprocessed in what way
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipeline = make_pipeline(preprocessor, RandomForestClassifier())
pipeline.fit(X_train,y_train)
pipeline.predict(X_test)

/home/wiesemann/.pyenv/versions/3.10.6/envs/animals/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array(['between 1 and 5 days', 'between 1 and 5 days',
       'higher than 30 days', ..., 'between 1 and 5 days',
       'between 1 and 5 days', 'higher than 30 days'], dtype=object)

In [56]:
pipeline.score(X_test,y_test)

0.44463737144383597

## TRAIN_TEST_DATASET- 2 classes

In [8]:
# I have to scale them? 

# Split the data into features (X) and target (y)

X = df[['intake_condition_2classes', 'sex', 'sex_type',\
    'animal_type', 'breed_2classes', 'age_upon_intake_(years)', 'color_3classes',\
    'outcome_type_2classes']]
y = df['time_in_shelter_days_round_2classes'] # next one with 'time_in_shelter_days_round_5classes', 'time_in_shelter_days_round_2classes'


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
# create a list of numerical columns and categorical columns
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = ['age_upon_intake_(years)']

categorical_features = ['intake_condition_2classes', 'sex', 'sex_type',\
    'animal_type', 'breed_2classes', 'color_3classes',\
    'outcome_type_2classes']

# create the transformer for numerical columns
numeric_transformer = make_pipeline(StandardScaler())

# create the transformer for categorical columns
categorical_transformer = make_pipeline(OneHotEncoder(sparse=False, handle_unknown='ignore'))


# use ColumnTransformer to specify which columns need to be preprocessed in what way
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipeline = make_pipeline(preprocessor, RandomForestClassifier())
pipeline.fit(X_train,y_train)
pipeline.predict(X_test)

/home/wiesemann/.pyenv/versions/3.10.6/envs/animals/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array(['one week', 'one week', 'more than one week', ...,
       'more than one week', 'one week', 'one week'], dtype=object)

In [10]:
pipeline.score(X_test,y_test)

0.7282623213570188

## TRAIN_TEST_DATASET- numerical

In [11]:
# I have to scale them? 

# Split the data into features (X) and target (y)

X = df[['intake_condition_2classes', 'sex', 'sex_type',\
    'animal_type', 'breed_2classes', 'age_upon_intake_(years)', 'color_3classes',\
    'outcome_type_2classes']]
y = df['time_in_shelter_days_round'] # next one with 'time_in_shelter_days_round_5classes', 'time_in_shelter_days_round_2classes'


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
# create a list of numerical columns and categorical columns
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_features = ['age_upon_intake_(years)']

categorical_features = ['intake_condition_2classes', 'sex', 'sex_type',\
    'animal_type', 'breed_2classes', 'color_3classes',\
    'outcome_type_2classes']

# create the transformer for numerical columns
numeric_transformer = make_pipeline(StandardScaler())

# create the transformer for categorical columns
categorical_transformer = make_pipeline(OneHotEncoder(sparse=False, handle_unknown='ignore'))


# use ColumnTransformer to specify which columns need to be preprocessed in what way
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipeline = make_pipeline(preprocessor, RandomForestClassifier())
pipeline.fit(X_train,y_train)
pipeline.predict(X_test)

/home/wiesemann/.pyenv/versions/3.10.6/envs/animals/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([4, 0, 1, ..., 1, 0, 4])

In [13]:
pipeline.score(X_test,y_test)

0.22438894083077335

## AUROC- for binary target

In [57]:
# Import the necessary libraries
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [61]:
# Create a datatset using the 'make_classification' function
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=0, random_state=42)
X = df[['intake_condition_2classes', 'sex', 'sex_type',\
    'animal_type', 'breed_2classes', 'age_upon_intake_(years)', 'color_3classes',\
    'outcome_type_2classes']]
y = df['time_in_shelter_days_round_2classes'] # next one with 'time_in_shelter_days_round_5classes', 'time_in_shelter_days_round_2classes'


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Split the dataset into training and testing set
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [62]:
# I HAVE TO CHOOSE MY PIPELINE???
# Train the RandomForestClassifier model on the training set
# rfc = RandomForestClassifier(n_estimators=100, random_state=42)
# rfc.fit(X_train, y_train)
pipeline.fit(X_train, y_train)

/home/wiesemann/.pyenv/versions/3.10.6/envs/animals/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  []),
                                                 ('cat',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['intake_condition_2classes',
                                                   'animal_type', 'sex',
                                                   'sex_type', 'breed_2classes',
                                                   'color_3classes',
                                                   'outcome_type_2classes'])])),
                ('randomforestclassifier', RandomForestClassifier())])

In [63]:
# predict the probabilities of the positive class (i.e., class 1) on the testing set
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

In [64]:
# compute the AUROC using the 'roc_auc_score' function
auroc = roc_auc_score(y_test, y_pred_proba)
print("AUROC score: {:.3f}".format(auroc))

AUROC score: 0.746


## GridSearch- hyperparameter tuning

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [66]:
# Create a datatset using the 'make_classification' function
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=0, random_state=42)
X = df[['intake_condition_2classes', 'sex', 'sex_type', 'animal_type', 'breed_2classes', 'age_upon_intake_(years)', 'color_3classes',\
    'outcome_type_2classes']]
y = df['time_in_shelter_days_round_2classes'] # next one with 'time_in_shelter_days_round_5classes', 'time_in_shelter_days_round_2classes'


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [67]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, X_train, y_train, cv=5)
mean_accuracy = scores.mean()

/home/wiesemann/.pyenv/versions/3.10.6/envs/animals/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/wiesemann/.pyenv/versions/3.10.6/envs/animals/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/wiesemann/.pyenv/versions/3.10.6/envs/animals/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/wiesemann/.pyenv/versions/3.10.6/envs/animals/lib/python3.10/

In [34]:
mean_accuracy

0.4815179917666155

In [35]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

NameError: name 'y_pred' is not defined

In [ ]:
#Hyperparameter Tuning
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)

## GridSearch

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('randomforestclassifier', RandomForestClassifier())
])

param_grid = {
    'randomforestclassifier__n_estimators': [50, 100, 200],
    'randomforestclassifier__max_depth': [3, 5, 7],
    'randomforestclassifier__max_features': [3, 5, 7, 10]
}

grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best hyperparameters: ", grid_search.best_params_)
print("Best accuracy score: {:.3f}".format(grid_search.best_score_))

ValueError: For multi-metric scoring, the parameter refit must be set to a scorer key or a callable to refit an estimator with the best parameter setting on the whole data and make the best_* attributes available for that metric. If this is not needed, refit should be set to False explicitly. True was passed.

In [ ]:
## target 2classes